### Youtube Comments Analysis

The dataset provided for this project are comments for videos related to animals and/or pets. The dataset is 240MB compressed; it was download from the below google drive link:
https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing

In [3]:
# link: https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing
# need to install googledrivedownloader==0.4 on cluster beforehand
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a', dest_path='./../../dbfs/laioffer/spark_hw3/data/animal_comments.gz')



In [4]:

%sh
gunzip -k /dbfs/laioffer/spark_hw3/data/animal_comments.gz
ls ../../dbfs/laioffer/spark_hw3/data/

gzip: /dbfs/laioffer/spark_hw3/data/animal_comments already exists;	not overwritten
animal_comments
animal_comments.csv
animal_comments.gz

In [5]:
%sh
cd /dbfs/laioffer/spark_hw3/data/

In this section, we download the data from googledrive using a google downloader and then unzip the file. The files are now in /dbfs/laioffer/spark_hw3/data/animal_comments

#### 0. Data Exploration and Cleaning

In this section, we are going to first do a preliminary exploration of the original dataset and drops all the null rows from the original dataset.

In [9]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("youtube analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [10]:
df_clean = spark.table("animal_comments")
df_clean.show(10)

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows

In [11]:
df_clean.count() 

Out[8]: 5820035

In [12]:
# Dropping rows containing null values.
df_clean = df_clean.na.drop(subset=["comment"])
df_clean.count()

Out[9]: 5818984

In [13]:
df_clean.show()

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟|
Hope For Paws - O...|2911.0| That mother cat l...|
Hope For Paws - O...|2911.0| Its people like H...|
 Talking Kitty Cat|2911.0| steve: No wet foo...|
 Brave Wilderness|3224.0| Dont call this a ...|
 MaxluvsMya|3267.0| why are you alway...|
Rise Up Society F...|3372.0| Deb Tucker|
 The Dodo|3466.0| Thats a deer isnt...|
 Brave Wilderness|3466.0| there is no safe ...|
 Brave Wilderness|3466.0| Red before yellow...|
+--------------------+------+-------------------------------------+
only showing top 20 rows

In [14]:
# find user with preference of dog and cat
from pyspark.sql.functions import when
from pyspark.sql.functions import col

# you can user your ways to extract the label

df_clean = df_clean.withColumn("label", \
                           (when(col("comment").like("%my dog%"), 1) \
                           .when(col("comment").like("%I have a dog%"), 1) \
                           .when(col("comment").like("%my cat%"), 1) \
                           .when(col("comment").like("%I have a cat%"), 1) \
                           .when(col("comment").like("%my puppy%"), 1) \
                           .when(col("comment").like("%my pup%"), 1) \
                           .when(col("comment").like("%my kitty%"), 1) \
                           .when(col("comment").like("%my pussy%"), 1) \
                           .otherwise(0)))

In [15]:
df_clean
display(df_clean)

creator_name,userid,comment,label
Doug The Pug,87.0,I shared this to my friends and mom the were lol,0
Doug The Pug,87.0,Super cute 😀🐕🐶,0
bulletproof,530.0,stop saying get em youre literally dumb . have some common sense or dont own this kind of dog. fucking retarded I swear,0
Meu Zoológico,670.0,Tenho uma jiboia e um largato,0
ojatro,1031.0,I wanna see what happened to the pigs after that please,0
Tingle Triggers,1212.0,Well shit now Im hungry,0
Hope For Paws - Official Rescue Channel,1806.0,when I saw the end it said to adopt I saw different animal sites I was mad that they separated the cute little pups after being together for a long time,0
Hope For Paws - Official Rescue Channel,2036.0,Holy crap. That is quite literally the most adorable pup Ive ever seen.,0
Life Story,2637.0,武器はクエストで貰えるんじゃないんですか？,0
Brian Barczyk,2698.0,Call the teddy Larry,0


In [16]:
print("have pet:", df_clean.filter(col('label') == 1.0).distinct().count())
print("no pet:", df_clean.filter(col('label') == 0.0).distinct().count())

have pet: 40110
no pet: 5717102

In the above cell, I mannually labeled the users with comments indicated that they are pet owners. And I find out the ration between pet owners and non pet owners are around 1:100(40110:5717102).

#### 1. Data preprocessing and Build the classifier

In the below section, I first used a regexTokenizer to tokenize and convert the sentence in the comments into words and remove the gaps. And then used Word2Vec to do a nueral words embeddings. The outputs are feature vectors.

In [20]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")

word2Vec = Word2Vec(inputCol="words", outputCol="features")

In [21]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, word2Vec])

# Fit the pipeline to training documents.
pipeline_fit = pipeline.fit(df_clean)
dataset = pipeline_fit.transform(df_clean)

In [22]:
dataset.show()

+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
 creator_name|userid| comment|label| words| features|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
 Doug The Pug| 87.0| I shared this to ...| 0|[i, shared, this,...|[0.01784441399980...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0| [super, cute]|[-0.2883199453353...|
 bulletproof| 530.0| stop saying get e...| 0|[stop, saying, ge...|[0.08585900381546...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|[tenho, uma, jibo...|[0.28823004026586...|
 ojatro|1031.0| I wanna see what ...| 0|[i, wanna, see, w...|[0.04281720959327...|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|[well, shit, now,...|[-0.0340470552444...|
Hope For Paws - O...|1806.0| when I saw the en...| 0|[when, i, saw, th...|[0.04950299204867...|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|[holy, crap, that...|[-0.0756140945909...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0| []| (100,[],[])|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|[call, the, teddy...|[0.07877420785371...|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟| 0| []| (100,[],[])|
Hope For Paws - O...|2911.0| That mother cat l...| 0|[that, mother, ca...|[-0.0103125133636...|
Hope For Paws - O...|2911.0| Its people like H...| 0|[its, people, lik...|[0.10304946278532...|
 Talking Kitty Cat|2911.0| steve: No wet foo...| 0|[steve, no, wet, ...|[0.22872448035261...|
 Brave Wilderness|3224.0| Dont call this a ...| 0|[dont, call, this...|[0.00677825643547...|
 MaxluvsMya|3267.0| why are you alway...| 0|[why, are, you, a...|[-0.0384437218308...|
Rise Up Society F...|3372.0| Deb Tucker| 0| [deb, tucker]|[0.15005647391080...|
 The Dodo|3466.0| Thats a deer isnt...| 0|[thats, a, deer, ...|[-0.0641004357021...|
 Brave Wilderness|3466.0| there is no safe ...| 0|[there, is, no, s...|[0.05535240537103...|
 Brave Wilderness|3466.0| Red before yellow...| 0|[red, before, yel...|[0.03447196202782...|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
only showing top 20 rows

In order to train and test the model, we need to first split the data into training and test datasets. However, in the previous section, we have already known that the data is very imbalance. We have very few rows labeled with 1, which is pet owners. And we have majority rows label with 0. The imbalance data will be a big obsticle in training classify models. In order to solve this problem, we are randomly undersample the data with 0 labels.

In [24]:
(lable1_train,lable1_test)=dataset.filter(col('label')==1).randomSplit([0.7, 0.3],seed = 100)
(lable0_train, lable0_ex)=dataset.filter(col('label')==0).randomSplit([0.01, 0.99],seed = 100)
(lable0_test, lable0_ex2)=lable0_ex.randomSplit([0.004, 0.996],seed = 100)


In [25]:
trainingData = lable0_train.union(lable1_train)
testData=lable0_test.union(lable1_test)

In [26]:
print("Dataset Count: " + str(dataset.count()))
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Dataset Count: 5818984
Training Dataset Count: 85977
Test Dataset Count: 34874

##### LogisticRegression

In the below section, we build a logistic regression model to classify the pet owner based on their youtube comments. We first trained the model using the trainning datas and testing datas. Then we used paramgridbuilder and crossvalidator packages to tune and validate the model.

In [29]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#build a logistic model 
lr = LogisticRegression(featuresCol="features",labelCol="label" )

#train model
lrModel = lr.fit(trainingData)

#tune the model with max iterator, regulation parameter and elastic net alpha
param_grid = ParamGridBuilder().addGrid(lr.maxIter,[5, 10]).addGrid(lr.regParam, [0.1, 0.01, 0.001]).addGrid(lr.elasticNetParam, [0.75, 0.8, 0.85]).build()

#set up the evaluator and define the proper metrics as AUC
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

#5-fold cross validation
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)

In [30]:
#fit the model and extract the best model
model = crossval.fit(trainingData)
best_model = model.bestModel

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [31]:
#use best model to predict the test data
predictions=best_model.transform(testData)
predictions.show(10)

+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 creator_name| userid| comment|label| words| features| rawPrediction| probability|prediction|
+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 Big Cat Rescue| 70179.0|One of them looke...| 1|[one, of, them, l...|[-0.0308075963985...|[-4.8858869485740...|[0.00749581065377...| 1.0|
 Big Cat Rescue|1813577.0|How do i grow my ...| 1|[how, do, i, grow...|[-0.0566847146385...|[-2.3191847440015...|[0.08954650322763...| 1.0|
 Big Cat Rescue|2477786.0|Definitely going ...| 1|[definitely, goin...|[-0.0014666513375...|[-0.7124683466845...|[0.32905365539143...| 1.0|
 Brandon|1500606.0|Yo are so lucky m...| 1|[yo, are, so, luc...|[-0.0060181027899...|[-0.2592637062043...|[0.43554471412682...| 1.0|
Brave Wilderness| 586033.0|lol that frog con...| 1|[lol, that, frog,...|[-0.1214310286566...|[-0.0213382902842...|[0.49466562983235...| 1.0|
Brave Wilderness| 947008.0|This k9 reminds m...| 1|[this, k9, remind...|[-0.0129154476408...|[-4.8364236876884...|[0.00787290823197...| 1.0|
Brave Wilderness| 971222.0|I wasnt sure if t...| 1|[i, wasnt, sure, ...|[0.00312346838472...|[-2.9363230757718...|[0.05038691636660...| 1.0|
Brave Wilderness|1400234.0|Thats the way I p...| 1|[thats, the, way,...|[0.00451156962662...|[-3.6337587436264...|[0.02573682194185...| 1.0|
Brave Wilderness|1777036.0|I have a dog name...| 1|[i, have, a, dog,...|[0.12423286317951...|[0.83454057648115...|[0.69731415398601...| 0.0|
Brave Wilderness|1847420.0|We have this liza...| 1|[we, have, this, ...|[0.04740891449761...|[-0.2297863540703...|[0.44280485734296...| 1.0|
+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows

In [32]:

print ("**Best Model**")
print (" Elastic:" +str(best_model._java_obj.parent().getElasticNetParam()))
print (" MaxIter:" +str(best_model._java_obj.parent().getMaxIter()))
print (" RegParam:" + str(best_model._java_obj.parent().getRegParam()))

**Best Model**
 Elastic:0.85
 MaxIter:10
 RegParam:0.001

In [33]:
training_summary = best_model.summary
training_summary.roc.show()

+--------------------+--------------------+
 FPR| TPR|
+--------------------+--------------------+
 0.0| 0.0|
0.001098881931241...|0.017553834126927723|
0.001517503619333345| 0.03688437211285623|
 0.0022849767141686| 0.05934190889062611|
0.002843138964957876| 0.07348447160827233|
0.003662939770804626| 0.09288607774856086|
0.004447855435977046| 0.10923175325136807|
 0.00525021367148663| 0.12404946343543458|
0.005930473914636061| 0.14060834340132186|
0.006610734157785491| 0.15467983796460805|
0.007308436971272087| 0.17223367209153578|
0.007936369503410022| 0.188650415748703|
0.008773612879593938| 0.2049960912515102|
0.009541085974429191| 0.22208798237509772|
0.010343444209938777| 0.23758084002558452|
 0.01145976871151733| 0.2558808897732926|
0.012506322931747223| 0.2719067585814796|
0.013413336589279796| 0.2907042854097079|
0.014302907676475206| 0.30662355198635494|
0.015541330170413913| 0.32414185203610263|
+--------------------+--------------------+
only showing top 20 rows

In [34]:
print("areaUnderROC(AUC): " + str(training_summary.areaUnderROC))

areaUnderROC(AUC): 0.9503392466506148

In [35]:
predictions.show(10)


+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 creator_name| userid| comment|label| words| features| rawPrediction| probability|prediction|
+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 Big Cat Rescue| 70179.0|One of them looke...| 1|[one, of, them, l...|[-0.0308075963985...|[-4.8858869485740...|[0.00749581065377...| 1.0|
 Big Cat Rescue|1813577.0|How do i grow my ...| 1|[how, do, i, grow...|[-0.0566847146385...|[-2.3191847440015...|[0.08954650322763...| 1.0|
 Big Cat Rescue|2477786.0|Definitely going ...| 1|[definitely, goin...|[-0.0014666513375...|[-0.7124683466845...|[0.32905365539143...| 1.0|
 Brandon|1500606.0|Yo are so lucky m...| 1|[yo, are, so, luc...|[-0.0060181027899...|[-0.2592637062043...|[0.43554471412682...| 1.0|
Brave Wilderness| 586033.0|lol that frog con...| 1|[lol, that, frog,...|[-0.1214310286566...|[-0.0213382902842...|[0.49466562983235...| 1.0|
Brave Wilderness| 947008.0|This k9 reminds m...| 1|[this, k9, remind...|[-0.0129154476408...|[-4.8364236876884...|[0.00787290823197...| 1.0|
Brave Wilderness| 971222.0|I wasnt sure if t...| 1|[i, wasnt, sure, ...|[0.00312346838472...|[-2.9363230757718...|[0.05038691636660...| 1.0|
Brave Wilderness|1400234.0|Thats the way I p...| 1|[thats, the, way,...|[0.00451156962662...|[-3.6337587436264...|[0.02573682194185...| 1.0|
Brave Wilderness|1777036.0|I have a dog name...| 1|[i, have, a, dog,...|[0.12423286317951...|[0.83454057648115...|[0.69731415398601...| 0.0|
Brave Wilderness|1847420.0|We have this liza...| 1|[we, have, this, ...|[0.04740891449761...|[-0.2297863540703...|[0.44280485734296...| 1.0|
+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows

In the above section, we have already train, test and tune the model. Next, we will evaluate the extracted best logistic model with accuracy, precision, recall and AUC.

In [37]:
TP = predictions.filter((col("label") == 1) & (col("prediction") == 1)).count()
FP = predictions.filter((col("label") == 0) & (col("prediction") == 1)).count()
TN = predictions.filter((col("label") == 0) & (col("prediction") == 0)).count()
FN = predictions.filter((col("label") == 1) & (col("prediction") == 0)).count()

accuracy = (TP + TN) / (TP + FP + TN + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print ("True Positives:", TP)
print ("False Positives:", FP)
print ("True Negatives:", TN)
print ("False Negatives:", FN)
print ("Test Accuracy:", accuracy)
print ("Test Precision:", precision)
print ("Test Recall:", recall)
print ("Test AUC of ROC:", training_summary.areaUnderROC)

True Positives: 9574
False Positives: 1588
True Negatives: 20959
False Negatives: 2509
Test Accuracy: 0.8816921744152469
Test Precision: 0.8577315893209102
Test Recall: 0.792352892493586
Test AUC of ROC: 0.9503392466506148

##### RandomForest

After using the logistic model, we then try to use the random forest model. We will repeat the same train test procedures as logistic model in this part.

In [40]:
from pyspark.ml.classification import RandomForestClassifier

#build a logistic model 
rf = RandomForestClassifier(labelCol="label" , featuresCol="features", numTrees=10)

#tune the model with max iterator, regulation parameter and elastic net alpha
param_grid = ParamGridBuilder().addGrid(rf.numTrees, [5, 10]).addGrid(rf.maxDepth, [3, 4, 5]).build()

#set up the evaluator and define the proper metrics as AUC
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

#5-fold cross validation
crossval = CrossValidator(estimator=rf, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

model = crossval.fit(trainingData)

#extract the best model
best_model = model.bestModel

#predict with test data
predictions=best_model.transform(testData)
predictions.show(10)

+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 creator_name| userid| comment|label| words| features| rawPrediction| probability|prediction|
+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 Big Cat Rescue| 70179.0|One of them looke...| 1|[one, of, them, l...|[-0.0308075963985...|[4.15968224502343...|[0.41596822450234...| 1.0|
 Big Cat Rescue|1813577.0|How do i grow my ...| 1|[how, do, i, grow...|[-0.0566847146385...|[4.09588383025993...|[0.40958838302599...| 1.0|
 Big Cat Rescue|2477786.0|Definitely going ...| 1|[definitely, goin...|[-0.0014666513375...|[3.77267155027606...|[0.37726715502760...| 1.0|
 Brandon|1500606.0|Yo are so lucky m...| 1|[yo, are, so, luc...|[-0.0060181027899...|[6.93680314927402...|[0.69368031492740...| 0.0|
Brave Wilderness| 586033.0|lol that frog con...| 1|[lol, that, frog,...|[-0.1214310286566...|[4.28763600947220...|[0.42876360094722...| 1.0|
Brave Wilderness| 947008.0|This k9 reminds m...| 1|[this, k9, remind...|[-0.0129154476408...|[2.68206984693582...|[0.26820698469358...| 1.0|
Brave Wilderness| 971222.0|I wasnt sure if t...| 1|[i, wasnt, sure, ...|[0.00312346838472...|[2.93420449781835...|[0.29342044978183...| 1.0|
Brave Wilderness|1400234.0|Thats the way I p...| 1|[thats, the, way,...|[0.00451156962662...|[3.63971498931157...|[0.36397149893115...| 1.0|
Brave Wilderness|1777036.0|I have a dog name...| 1|[i, have, a, dog,...|[0.12423286317951...|[4.38984946343491...|[0.43898494634349...| 1.0|
Brave Wilderness|1847420.0|We have this liza...| 1|[we, have, this, ...|[0.04740891449761...|[2.68206984693582...|[0.26820698469358...| 1.0|
+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows

In [41]:
print ("**Best Model**")
print (" numTree:" +str(best_model._java_obj.parent().getNumTrees()))
print (" RegParam:" + str(best_model._java_obj.parent().getMaxDepth()))

**Best Model**
 numTree:10
 RegParam:5

In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
AUC = evaluator.evaluate(predictions)

In [43]:
TP = predictions.filter((col("label") == 1) & (col("prediction") == 1)).count()
FP = predictions.filter((col("label") == 0) & (col("prediction") == 1)).count()
TN = predictions.filter((col("label") == 0) & (col("prediction") == 0)).count()
FN = predictions.filter((col("label") == 1) & (col("prediction") == 0)).count()

accuracy = (TP + TN) / (TP + FP + TN + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print ("True Positives:", TP)
print ("False Positives:", FP)
print ("True Negatives:", TN)
print ("False Negatives:", FN)
print ("Test Accuracy:", accuracy)
print ("Test Precision:", precision)
print ("Test Recall:", recall)
print ("Test AUC of ROC:", AUC)

True Positives: 8620
False Positives: 1519
True Negatives: 21028
False Negatives: 3463
Test Accuracy: 0.8561362980075079
Test Precision: 0.8501824637538219
Test Recall: 0.7133989903169743
Test AUC of ROC: 0.9257309735602243

##### Gradient boosting

In this part, we will try to train a Gradiant boosting tree model.

In [46]:
from pyspark.ml.classification import GBTClassifier

GDBT= GBTClassifier(labelCol="label" , featuresCol="features")


param_grid = ParamGridBuilder().addGrid(GDBT.stepSize, [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]).addGrid(GDBT.maxDepth, [3, 4, 5]).build()

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

crossval = CrossValidator(estimator=GDBT,
                          estimatorParamMaps=param_grid,
                          evaluator=evaluator,
                          numFolds=5)
model = crossval.fit(trainingData)
best_model = model.bestModel
predictions=best_model.transform(testData)
predictions.show(10)

+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 creator_name| userid| comment|label| words| features| rawPrediction| probability|prediction|
+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 Big Cat Rescue|1750422.0|Hes so handsome.....| 1|[hes, so, handsom...|[-0.0364790500607...|[-1.0227899237879...|[0.11449977558945...| 1.0|
 Brandon|1500606.0|Yo are so lucky m...| 1|[yo, are, so, luc...|[-0.0060181027899...|[0.60629151193732...|[0.77075566623130...| 0.0|
Brave Wilderness| 346987.0|my dog got bitten...| 1|[my, dog, got, bi...|[-0.1339863979003...|[-0.9200076919887...|[0.13704947314974...| 1.0|
Brave Wilderness| 349705.0|Me: *Watches this...| 1|[me, watches, thi...|[0.00758161162957...|[-0.7309692449946...|[0.18817101725852...| 1.0|
Brave Wilderness| 947008.0|This k9 reminds m...| 1|[this, k9, remind...|[-0.0129154476408...|[-2.1368445117505...|[0.01373891189680...| 1.0|
Brave Wilderness|1400234.0|Thats the way I p...| 1|[thats, the, way,...|[0.00451156962662...|[-2.1305727899721...|[0.01390991828340...| 1.0|
Brave Wilderness|1477239.0|Coyote because it...| 1|[coyote, because,...|[-0.1444989261550...|[-0.7469024005832...|[0.18335133424488...| 1.0|
Brave Wilderness|1777036.0|I have a dog name...| 1|[i, have, a, dog,...|[0.12423286317951...|[0.49319328695109...|[0.72837361251485...| 0.0|
 Brian Barczyk| 127220.0|I know how it fee...| 1|[i, know, how, it...|[-0.0399990759841...|[-1.3194465771228...|[0.06667688274977...| 1.0|
 Brian Barczyk| 138593.0|Ah man my dog has...| 1|[ah, man, my, dog...|[-0.0035606055340...|[-0.3681599142980...|[0.32380941954862...| 1.0|
+----------------+---------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows

In [47]:
print ("**Best Model**")
print (" learning rate:" +str(best_model._java_obj.parent().getStepSize()))
print (" max depth:" + str(best_model._java_obj.parent().getMaxDepth()))

**Best Model**
 learning rate:0.3
 max depth:5

In [48]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
AUC = evaluator.evaluate(predictions)

In [49]:
TP = predictions.filter((col("label") == 1) & (col("prediction") == 1)).count()
FP = predictions.filter((col("label") == 0) & (col("prediction") == 1)).count()
TN = predictions.filter((col("label") == 0) & (col("prediction") == 0)).count()
FN = predictions.filter((col("label") == 1) & (col("prediction") == 0)).count()

accuracy = (TP + TN) / (TP + FP + TN + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print ("True Positives:", TP)
print ("False Positives:", FP)
print ("True Negatives:", TN)
print ("False Negatives:", FN)
print ("Test Accuracy:", accuracy)
print ("Test Precision:", precision)
print ("Test Recall:", recall)
print ("Test AUC of ROC:", AUC)

True Positives: 10581
False Positives: 1383
True Negatives: 21187
False Negatives: 1474
Test Accuracy: 0.9174873646209386
Test Precision: 0.8844032096288866
Test Recall: 0.8777270841974284
Test AUC of ROC: 0.9720598655512402

Since the best Gradient Boosting model has a test AUC of 0.97, the highest among all the three models. We choose the best Gradient Boosting model with learning rate of 0.3 and max iteration of 5.

#### 2. Classify All The Users

In this section, since the users labeled 1 are those we are sure that they are pet onwers. And the users labeled 0 are the one did not indicate they are onwers. They probabely are not pet onwers but we are not sure. So we put datas labled 0 into the best model to classify those users. After the classification, we then combine the newly classified set and the previously labled set to get all the pet onwers in the dataset.

In [53]:
df_all0 = dataset.filter((col("label") == 0))
display(df_all0)

creator_name,userid,comment,label,words,features
Doug The Pug,87.0,I shared this to my friends and mom the were lol,0,"List(i, shared, this, to, my, friends, and, mom, the, were, lol)","List(1, 100, List(), List(0.01784441399980675, -0.020379101772877304, 0.12678783395412294, -0.03029621936465529, -0.017744215416975996, 0.022211988710544327, -0.11156877909194339, 0.1452448835427111, 0.11735361069440842, 0.025073884224349804, -0.10923891103911129, 0.018014919690110466, -0.08526893506165255, -0.11025032222228633, -0.11347918474877423, -0.009192273698069834, -0.057526062581349506, 0.061800442043353214, -0.12616047330877997, -0.058853128416971726, 0.007348398805003275, 0.015894510867920788, 0.03985926652835174, -0.04098683371293274, 0.0723224056260237, -0.05881796857680787, 0.10799326906404035, 0.08547606941481883, -0.034434194189750335, 0.050004090055484667, 0.0025668262758038263, 0.05669034407897429, 0.019723338244313545, 0.01534969667607749, -0.01758848630230535, -0.06886818903413686, 0.056710246950387955, 0.036821977997368034, -0.048310428519140594, 0.21720414032990282, -0.12300450515679338, -0.04557176570365713, -0.02733937947248871, -0.020434588532556187, 0.093020345677029, 0.011690069836649029, 0.030635233291170814, -0.13460135779512877, 0.020283695500852034, 0.016415423967621544, -0.11034134131940929, -0.19184056059880691, -0.1208180675113743, 0.07561470805244012, 0.24518262865868481, -0.07774396096779541, 0.07317259772257372, -0.20141763443296606, 0.019589733671058308, 0.004427224820987745, 0.06102355414027856, 0.04739846458489245, 0.16153187728063628, 0.03162644363262437, -0.012427768077362667, 0.006310680373148485, 0.13922911255874418, 0.06705699482170696, 0.11559060859409245, 0.03718750894239003, 0.034705567554655405, -0.0036969812912998764, 0.14047886058688164, -0.010464219884438948, -0.13436831580474973, -0.01427018828690052, 0.05217442593791268, 0.20534982532262802, 0.0060194998302242975, -0.007256170819428834, 0.0036896970123052597, -0.03589680528437549, 0.027363426475362346, -0.0011968112296678803, -0.03598377921364524, 0.06176092894747853, -0.03407896360890432, -0.032485040904165624, -0.14415372704917734, 0.09035833958875049, 0.03175183148546652, -0.11052434180270541, -0.1539065070788969, 0.02812724154103886, 0.038774443472820254, 0.054216154935685074, -6.559804420579563E-4, -0.06792547320947051, -0.04848683320663192, 0.031643440227278254))"
Doug The Pug,87.0,Super cute 😀🐕🐶,0,"List(super, cute)","List(1, 100, List(), List(-0.2883199453353882, -0.03971422091126442, 0.26716596633195877, 0.17103654332458973, 0.2350746262818575, 0.10617415606975555, 0.30452148616313934, 0.17845770105486736, -0.023327077738940716, 0.300587996840477, -0.2319678533822298, 0.22827358543872833, 0.2698281481862068, 0.1508161723613739, -0.1575203153770417, 0.04653379041701555, -0.016199510544538498, -0.2715160623192787, 0.07917454093694687, 0.171640757471323, 0.08451637253165245, 0.4065053462982178, 0.31024640426039696, -0.1791418492794037, -0.3042733445763588, -0.11411438323557377, 0.17948328703641891, -0.1565072238445282, 0.14295890927314758, -0.002457171678543091, -0.1484678965061903, 0.41716181486845016, -0.16310758143663406, -0.3879905492067337, 0.10439009498804808, 0.0014057625085115433, 0.2724960520863533, -0.0018823854625225067, 0.08369084633886814, 0.1448155753314495, -0.03386777639389038, 0.11670710518956184, -0.0755307525396347, 0.052051153033971786, -0.08758007176220417, -0.02676476538181305, 0.08695698529481888, -0.11278240010142326, 0.4293353706598282, -0.13709503412246704, -0.050914641469717026, 0.12392759881913662, -0.045077405869960785, -0.0732060894370079, 0.12585338577628136, 0.08409356884658337, 0.2712007239460945, 0.10406625643372536, 0.20672351121902466, 0.13562875613570213, 0.24732454866170883, 0.07576844841241837, -0.04651647061109543, 0.011338777840137482, 0.2172185704112053, -0.09791882336139679, -0.0460000354796648, 0.21088873967528343, 0.044737400487065315, -0.18638974241912365, -0.06498082354664803, 0.2066710293

In [54]:
pred_all = best_model.transform(df_all0)
pred_all.show(10)

+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 creator_name|userid| comment|label| words| features| rawPrediction| probability|prediction|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
 Doug The Pug| 87.0| I shared this to ...| 0|[i, shared, this,...|[0.01784441399980...|[-0.2139104799268...|[0.39464678609912...| 1.0|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0| [super, cute]|[-0.2883199453353...|[2.06003040635659...|[0.98401610817305...| 0.0|
 bulletproof| 530.0| stop saying get e...| 0|[stop, saying, ge...|[0.08585900381546...|[0.79980281245013...|[0.83196325855699...| 0.0|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|[tenho, uma, jibo...|[0.28823004026586...|[1.98000898334994...|[0.98129381984989...| 0.0|
 ojatro|1031.0| I wanna see what ...| 0|[i, wanna, see, w...|[0.04281720959327...|[0.71086766710432...|[0.80561031792597...| 0.0|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|[well, shit, now,...|[-0.0340470552444...|[1.99522856940094...|[0.98184445968497...| 0.0|
Hope For Paws - O...|1806.0| when I saw the en...| 0|[when, i, saw, th...|[0.04950299204867...|[0.05578870418170...|[0.52786544880351...| 0.0|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|[holy, crap, that...|[-0.0756140945909...|[1.22279185736109...|[0.92023789599561...| 0.0|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0| []| (100,[],[])|[1.96049375305845...|[0.98056374473863...| 0.0|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|[call, the, teddy...|[0.07877420785371...|[1.39573949954246...|[0.94221362463705...| 0.0|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 10 rows

In [55]:

#number of total user
total_user = dataset.select('userid').distinct().count()
#number of labeled owner
owner_labeled = dataset.filter(col('label') == 1.0).distinct().count() 
#number of owner predicted
owner_pred = pred_all.filter(col('prediction') == 1.0).distinct().count()

fraction = (owner_labeled+owner_pred)/total_user
print('Fraction of the users who are cat/dog owners (ML estimate): ', round(fraction,3))

Fraction of the users who are cat/dog owners (ML estimate): 0.158

#### 3. Find out the topics important to the pet onwers.

In this section, I want to find out the hidden topics that are mostly important to pet owners. In order to do that, we need to use a Latent Dirichlet allocation (LDA) model. Before we train the model, we are going to do some data preprocessing. Removing all the stop words from the texts.

In [58]:
from pyspark.ml.feature import StopWordsRemover

find_topics= dataset.filter(col('label') == 1.0)
display(find_topics)


creator_name,userid,comment,label,words,features
The Dodo,24401.0,Now I want to try that with my dog!!!,1,"List(now, i, want, to, try, that, with, my, dog)","List(1, 100, List(), List(0.056104992619819105, 0.050285025499761105, 0.11073548977987634, -0.04980612132284376, 0.07449254652278291, 0.0707605845398373, -0.1215410472618209, -0.0019604708585474226, 0.012262929230928421, -0.023461801103419725, -0.09825960174202919, 0.017990300224887, -0.05660707738974856, -0.05604188113162915, -0.0530484484301673, -0.030050284746620387, -0.06695526838302612, 0.09616125592341025, -0.12502048992448384, 0.02976697931687037, 0.03735411746634377, 0.0725842545636826, -0.011665587986095084, -0.004889869855509863, 0.09053938608202669, -0.09636249429442816, 0.1071703533331553, -0.06528256088495255, -0.044271935100874134, 0.010242050720585717, -0.009210775088932779, 0.10560674137539333, 0.06459492072463036, 0.04222452330092589, -0.12644597536159885, 0.02462833643787437, -0.006274375236696667, -0.0874204476777878, -0.07095742308431201, 0.1777695222861237, -0.12643878327475652, -0.07588901743292809, 0.08988336670315927, 0.0565632503065798, 0.04414659324619505, -0.028848238910237946, -0.03874571041928397, 0.03944606220142709, -0.018145894217822287, 0.02805910880366961, -0.029242295771837234, -0.20419202082686952, -0.09253901884787612, 0.01294391933414671, 0.1389701979027854, -0.1819789818384581, 0.06204639095813036, -0.11334567310081588, 0.13792605987853473, -0.02634068847530418, 0.05851504881219524, -0.015389719977974892, 0.04392962530255318, -0.07011616147226757, 0.010950049488908714, -0.03974952274519536, 0.22382714288930097, 0.10857764010628064, 0.04235041982287334, -0.002972521896784504, -0.03819861407909128, 0.0015110371427403556, 0.21475149856673345, -0.006662546657025814, -0.09540232224389911, -0.07881565397191378, 0.03798630378312535, 0.06034875247213575, 0.027262467063135568, -6.380845508020785E-4, 0.10569482472621732, -0.13261165635453329, 0.03771849597493807, 0.0350563522014353, 0.07201329411731826, 0.054166774161987834, 0.05912539611260096, -0.04117297815779845, -0.07249593155251609, 0.04673711996939447, 2.5771144363615247E-5, -0.18946548447840741, -0.14548150677647856, 0.08335085461537042, 0.1571948150069349, 0.07877916180425219, 0.040688058050970234, -0.047293868381530046, 0.05456212920964592, -0.019719206650430955))"
Cole & Marmalade,43112.0,I blow smoke in my cats ear right to his brain,1,"List(i, blow, smoke, in, my, cats, ear, right, to, his, brain)","List(1, 100, List(), List(0.08869852024045857, -0.06921172980219126, 0.18917915267361837, -0.08392188867384738, -0.0724193842404268, 0.10376042754135349, -0.058406558226455345, 0.0159420735118064, 0.05600973074350887, 0.01039572474970059, -0.04562356373803182, 0.030906513693149795, 0.0059270535553382206, -0.04341583893719045, -0.17895731431516734, -0.004143979188732125, -0.03654006860134277, 0.03178462005135688, -0.10622292587702925, -0.04558534479954026, -0.013947658748789267, 0.06744514371861111, 0.02970695360140367, -0.0164144096726721, 0.06996051780879498, 0.024682228749787267, -0.07928733188997615, 0.046117245304313575, 0.023222364273599604, 0.06468839943408966, -0.002066387710246173, 0.08365636691451073, 0.012941054957495495, 0.09589683203111318, -0.06967235153371638, -9.155168452046134E-4, 0.03265694935213436, 0.01826491951942444, -0.005401689220558514, 0.20245216143402187, 0.01650929484855045, 0.060834553760783325, 0.134439828382297, -0.052835835281505504, 0.18231407502158123, 0.07378573647954248, -0.04818564484065229, 0.022946046749976547, -0.10144006308506835, 0.0026925958439030433, -0.12242365831678564, -0.19196268285370685, -0.08273805415427143, 0.04770046032287858, 0.08549785512414845, -0.11029721813445742, -0.014890094139528546, -0.20521191278980538, 0.10971558762883599, 0.038205173882571136, 0.03480411214034327, -0.013642556457356974, 0.027942037040537056, -0.01652187896384434, 0.001935300501910123, -0.029198415132916787, 0.2588126098906452, 0.04059295136142861, -0.03442

In [59]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
removed = remover.transform(find_topics)
display(removed)

creator_name,userid,comment,label,words,features,filtered
The Dodo,24401.0,Now I want to try that with my dog!!!,1,"List(now, i, want, to, try, that, with, my, dog)","List(1, 100, List(), List(0.056104992619819105, 0.050285025499761105, 0.11073548977987634, -0.04980612132284376, 0.07449254652278291, 0.0707605845398373, -0.1215410472618209, -0.0019604708585474226, 0.012262929230928421, -0.023461801103419725, -0.09825960174202919, 0.017990300224887, -0.05660707738974856, -0.05604188113162915, -0.0530484484301673, -0.030050284746620387, -0.06695526838302612, 0.09616125592341025, -0.12502048992448384, 0.02976697931687037, 0.03735411746634377, 0.0725842545636826, -0.011665587986095084, -0.004889869855509863, 0.09053938608202669, -0.09636249429442816, 0.1071703533331553, -0.06528256088495255, -0.044271935100874134, 0.010242050720585717, -0.009210775088932779, 0.10560674137539333, 0.06459492072463036, 0.04222452330092589, -0.12644597536159885, 0.02462833643787437, -0.006274375236696667, -0.0874204476777878, -0.07095742308431201, 0.1777695222861237, -0.12643878327475652, -0.07588901743292809, 0.08988336670315927, 0.0565632503065798, 0.04414659324619505, -0.028848238910237946, -0.03874571041928397, 0.03944606220142709, -0.018145894217822287, 0.02805910880366961, -0.029242295771837234, -0.20419202082686952, -0.09253901884787612, 0.01294391933414671, 0.1389701979027854, -0.1819789818384581, 0.06204639095813036, -0.11334567310081588, 0.13792605987853473, -0.02634068847530418, 0.05851504881219524, -0.015389719977974892, 0.04392962530255318, -0.07011616147226757, 0.010950049488908714, -0.03974952274519536, 0.22382714288930097, 0.10857764010628064, 0.04235041982287334, -0.002972521896784504, -0.03819861407909128, 0.0015110371427403556, 0.21475149856673345, -0.006662546657025814, -0.09540232224389911, -0.07881565397191378, 0.03798630378312535, 0.06034875247213575, 0.027262467063135568, -6.380845508020785E-4, 0.10569482472621732, -0.13261165635453329, 0.03771849597493807, 0.0350563522014353, 0.07201329411731826, 0.054166774161987834, 0.05912539611260096, -0.04117297815779845, -0.07249593155251609, 0.04673711996939447, 2.5771144363615247E-5, -0.18946548447840741, -0.14548150677647856, 0.08335085461537042, 0.1571948150069349, 0.07877916180425219, 0.040688058050970234, -0.047293868381530046, 0.05456212920964592, -0.019719206650430955))","List(want, try, dog)"
Cole & Marmalade,43112.0,I blow smoke in my cats ear right to his brain,1,"List(i, blow, smoke, in, my, cats, ear, right, to, his, brain)","List(1, 100, List(), List(0.08869852024045857, -0.06921172980219126, 0.18917915267361837, -0.08392188867384738, -0.0724193842404268, 0.10376042754135349, -0.058406558226455345, 0.0159420735118064, 0.05600973074350887, 0.01039572474970059, -0.04562356373803182, 0.030906513693149795, 0.0059270535553382206, -0.04341583893719045, -0.17895731431516734, -0.004143979188732125, -0.03654006860134277, 0.03178462005135688, -0.10622292587702925, -0.04558534479954026, -0.013947658748789267, 0.06744514371861111, 0.02970695360140367, -0.0164144096726721, 0.06996051780879498, 0.024682228749787267, -0.07928733188997615, 0.046117245304313575, 0.023222364273599604, 0.06468839943408966, -0.002066387710246173, 0.08365636691451073, 0.012941054957495495, 0.09589683203111318, -0.06967235153371638, -9.155168452046134E-4, 0.03265694935213436, 0.01826491951942444, -0.005401689220558514, 0.20245216143402187, 0.01650929484855045, 0.060834553760783325, 0.134439828382297, -0.052835835281505504, 0.18231407502158123, 0.07378573647954248, -0.04818564484065229, 0.022946046749976547, -0.10144006308506835, 0.0026925958439030433, -0.12242365831678564, -0.19196268285370685, -0.08273805415427143, 0.04770046032287858, 0.08549785512414845, -0.11029721813445742, -0.014890094139528546, -0.20521191278980538, 0.10971558762883599, 0.038205173882571136, 0.03480411214034327, -0.013642556457356974, 0.027942037040537056, -0.01652187896384434, 0.001935300501910123, -0.029198415132916787, 0.258812609890645

The CountVectorizer provides a simple way to both tokenize a collection of text documents and build a vocabulary of known words, but also to encode new documents using that vocabulary. We used the vector it generate to fit the LDA models. Since our goal is to find topics from the texts, after train the LDA model. We extract the topics from the model and use a user difined function to translate the words in the topics into understandable english.

In [61]:
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel

removed = removed.select('userid','filtered')

# minTF, minium number of times a word must appear in a document
# minDF, minimun number of documents a word must appear in
cv = CountVectorizer(inputCol="filtered", outputCol="features",minTF=2, minDF=4) 

countVectorModel = cv.fit(removed)

countVectors = (countVectorModel .transform(removed).select("userid", "features").cache())

print(len(countVectorModel.vocabulary))  # how many documents, vocab size

# k, number of topics
lda = LDA(k = 10, maxIter = 50 )


ldaModel = lda.fit(countVectors)


# Print topics and top-weighted terms
topics = ldaModel.describeTopics(maxTermsPerTopic=20)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)




9059
The topics described by their top-weighted terms:
+-----+---------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
topic|termIndices |termWeights |
+-----+---------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
0 |[1, 4, 3, 11, 41, 6, 125, 272, 187, 161, 37, 114, 155, 60, 27, 202, 95, 181, 137, 89] |[0.23110806770114553, 0.0812224441646646, 0.04089555568412669, 0.025197076406995568, 0.02353911022637408, 0.0188063434942907, 0.009021514733558956, 0.008700776854740732, 0.00830608967699174, 0.0081909578845569, 0.00704452920542248, 0.006359139697041459, 0.006012468905553507, 0.005603489133446005, 0.0053399413661810525, 0.005316532493077039, 0.004905949792339149, 0.004524939461466618, 0.0041846114748547445, 0.003593234651084879] |
1 |[72, 262, 954, 175, 271, 1062, 600, 690, 219, 1302, 536, 789, 824, 3474, 213, 786, 1115, 1063, 380, 1961]|[0.055326908481011766, 0.013766583787917062, 0.012280545937403195, 0.009783172539359543, 0.00788744063848555, 0.007050486703068505, 0.006833577087998343, 0.006745809129526443, 0.006197954028448303, 0.0060531508938365414, 0.005375213024635048, 0.005345377065995193, 0.004994048852420122, 0.004829260154050598, 0.004654628082402304, 0.0044092773481735036, 0.0041369123979655194, 0.004036976951200921, 0.0039084520028123465, 0.003840089100869809] |
2 |[32, 257, 54, 0, 358, 215, 457, 592, 118, 584, 303, 948, 82, 647, 3, 426, 501, 465, 285, 638] |[0.03412569117820063, 0.030830450868531954, 0.01871286901117817, 0.01800603115582674, 0.012930367854972728, 0.010924770807014277, 0.010448495966208933, 0.008743474069054463, 0.007722939847015908, 0.007546845922197057, 0.00736622277320258, 0.007017269913615832, 0.006733885649190572, 0.006683951749921851, 0.0066654784516275755, 0.00654462362430285, 0.006484823494096902, 0.00618202031444299, 0.006033212119816335, 0.005582196869609019] |
3 |[0, 2, 3, 7, 8, 9, 20, 10, 5, 6, 12, 14, 68, 16, 13, 18, 23, 19, 51, 25] |[0.13885860614807577, 0.03531001035404688, 0.02508643344563473, 0.01934981366658463, 0.013416323013547688, 0.013017640676968098, 0.012400041123931472, 0.01086082948369997, 0.010237305313178485, 0.009557336489668966, 0.009419196969990596, 0.008836546558176143, 0.007092860707131936, 0.006974915743129345, 0.0067957551213600345, 0.006698450524925645, 0.006560677550601492, 0.006531030174355976, 0.006247026727778905, 0.005855096787369341] |
4 |[28, 33, 239, 473, 146, 255, 697, 281, 191, 530, 209, 1084, 148, 282, 1271, 551, 500, 359, 462, 308] |[0.025032333342984152, 0.021603018763748497, 0.015840818262886268, 0.012081374449355113, 0.01024760454220517, 0.010242664560793247, 0.009316190064306274, 0.008915916488914951, 0.008863285749741455, 0.006508625558673767, 0.006239577537265291, 0.005345522673194619, 0.005336428118104062, 0.004913893694444402, 0.004777559804975469, 0.004595565470905003, 0.004593598557882498, 0.004358347062722938, 0.004289641826937185, 0.0038603770324172524] |
5 |[34, 45, 88, 422, 377, 104, 217, 517, 526, 625, 1016, 246, 611, 230, 470, 920, 1064, 737, 1548, 987] |[0.04973

In [62]:
transformed = ldaModel.transform(countVectors)
transformed.show(truncate=False)

+--------+--------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
userid |features |topicDistribution |
+--------+--------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
24401.0 |(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
43112.0 |(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
63541.0 |(9059,[107,624],[3.0,2.0]) |[0.8601178318463095,0.013221566708208695,0.013593801874176931,0.02989836889256276,0.013252549273574474,0.013577545058902648,0.013252621525776968,0.015202399357771089,0.014449720362606487,0.013433595100110305]|
175184.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
203881.0|(9059,[7],[3.0]) |[0.02933238986374573,0.019926908942209748,0.020487855896738738,0.8049055188913578,0.01997363277229795,0.020463391873859517,0.019973703124831068,0.022912153256305117,0.021777977418160222,0.02024646796049408] |
263721.0|(9059,[2,7,56,417,609,787,1296],[4.0,4.0,2.0,3.0,2.0,2.0,2.0])|[0.005800852779456446,0.0039404432728458345,0.11573574883458543,0.6627273289410929,0.1069985385614996,0.004046387347209996,0.08791023272324297,0.004530645415793534,0.004306335557978274,0.004003486566294987] |
273292.0|(9059,[1],[2.0]) |[0.7187356802518476,0.026696471442335862,0.027448000841110383,0.0591897043502537,0.02675902920267452,0.027415239430970264,0.02675916883877443,0.0306957802547839,0.02917634128008719,0.027124584107161887] |
329273.0|(9059,[4],[2.0]) |[0.7187338358097535,0.026696473183863836,0.02744800209621755,0.05919150988666588,0.02675902908947344,0.027415228253638284,0.026759171302369853,0.0306957882561511,0.029176376115808738,0.02712458600605793] |
344406.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
346294.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
381445.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
424882.0|(9059,[1],[2.0]) |[0.7187356802518476,0.026696471442335862,0.027448000841110383,0.0591897043502537,0.02675902920267452,0.027415239430970264,0.02675916883877443,0.0306957802547839,0.02917634128008719,0.027124584107161887] |
497349.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
497611.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
542233.0|(9059,[3],[3.0]) |[0.7895701542670153,0.01992690720895457,0.020487980660127834,0.04466749206135758,0.019973597326699816,0.02046339157154002,0.01997369762237417,0.022912406669591802,0.021777910073341563,0.020246462538997492] |
578367.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
598125.0|(9059,[6],[2.0]) |[0.039311963632506726,0.026696471887239382,0.02744800341933041,0.05933202566874759,0.026759036995199534,0.02741522933823366,0.02675917713995728,0.030695837649775154,0.7084576689995998,0.027124585269410455] |
689464.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
711033.0|(9059,[1,12,29],[2.0,2.0,2.0]) |[0.325975770535006,0.011317436335706546,0.011636023293337875,0.5798800058276642,0.011343950361491353,0.01162213703747752,0.011344013517103879,0.013013015856168732,0.012368719056256475,0.011498928179787697] |
769264.0|(9059,[],[]) |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |
+--------+--------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 20 rows

In [63]:
vocabArray = countVectorModel.vocabulary

ListOfIndexToWords = udf(lambda wl: list([vocabArray[w] for w in wl]))
FormatNumbers = udf(lambda nl: ["{:1.4f}".format(x) for x in nl])

topics.select(ListOfIndexToWords(topics.termIndices).alias('Words list in Top 10 Topics ')).show(truncate=False, n=10)


+------------------------------------------------------------------------------------------------------------------------------------------------+
Words list in Top 10 Topics |
+------------------------------------------------------------------------------------------------------------------------------------------------+
[cat, cats, dogs, got, food, one, kitten, raw, water, kittens, years, named, black, ive, old, treats, found, feed, baby, mom] |
[please, d, cream, cry, kill, ice, marmalade, poop, 10, turtle, nose, gibbyson, hole, whip, times, butt, milk, losing, morning, stung] |
[lol, pit, thats, dog, fish, beautiful, bull, tree, thanks, hamster, german, bulls, ever, o, dogs, shepherd, pups, pitbull, hand, bc] |
[dog, like, dogs, im, get, dont, people, know, love, one, time, really, u, want, much, go, never, even, animals, back] |
[good, away, girl, max, omg, treat, names, boy, passed, paw, ill, rottweiler, stop, remember, fireworks, fuck, favorite, high, hair, hit] |
[videos, cute, watching, ball, catnip, pet, today, sylvester, shelby, belly, boo, husky, snakes, pup, bird, steve, gibson, smell, spider, grass]|
[vet, lost, surgery, eye, way, horses, walk, know, look, horse, around, done, kitchen, teach, hold, nip, hear, zak, test, mochi] |
[hes, kitty, name, started, house, little, 3, see, dad, play, bed, leave, used, new, night, outside, tried, didnt, face, cancer] |
[love, puppy, one, help, snake, 1, door, snow, xd, talk, toys, animals, family, meow, toy, bites, half, knows, cat, fetch] |
[get, thank, birthday, great, year, rabbit, side, page, table, road, attention, give, cheese, super, mr, smokey, along, brian, week, voice] |
+------------------------------------------------------------------------------------------------------------------------------------------------+

Since the model only do a clustering of the datas, it finds different topics with word lists. I, as a humman, intepreate the top 4 topics from the words listed in the topics.

In [65]:
print("**Top 4 topics**")
print("cats with babies")
print("breakfast")
print("dogs with farm")
print("dogs friendly with people or other animals")

**Top 4 topics**
cats with babies
breakfast
dogs with farm
dogs friendly with people or other animals

#### 4. Identify Creators With Cat And Dog Owners In The Audience

In [67]:
dataset.createOrReplaceTempView("dataset_sql")

In [68]:
%sql
Select creator_name, count(userid) From dataset_sql Where label = 1.0 Group by creator_name Order by count(userid) desc limit 10


creator_name,count(userid)
The Dodo,4110
Cole & Marmalade,2904
Gohan The Husky,2353
Zak Georges Dog Training rEvolution,2212
Hope For Paws - Official Rescue Channel,1873
Vet Ranch,1748
Gone to the Snow Dogs,1718
Brian Barczyk,1609
Robin Seplut,1584
Taylor Nicole Dean,1575


###5. Analysis and Future work

In this project, we used three different classification models to classify the pet owners from youtube comments. In this case, the gradient boosting model out perform slightly to the other two with AUC of 0.97. Based on the classifier, around 15 percent of all the users from the datasets are pet onwers. We also used LDA model to find out the topics pet onwer interested most are "cats with babies", "breakfast", "dogs with farm", and "dogs friendly with people or other animals". 

Some problems I encountered during the project:
1.  The prediction result of 15 percent pet onwers in all the users seems too high, compared with 1 percent labled onwers in the original dataset. The reason is becuase when we first train the model with used undersampling to due with the imbalance data with a precission rate of 0.88. It means there are 0.12 chance we will falsely predict non pet onwers as pet onwers. Since the original dataset includes too many non pet onwers(labled 0), when we through all the data into the model, it falsely predict too many non pet onwers as pet oners. 
2.  The users that we labled 0 are the users who did not say they have pets or indicates they have pets. We actually cannot sure they don't have pets. Using them to train the classifiers may impact the true accuracy of the classifiers.
3.  How to better intepreate the topics extracts from LDA models. Since LDA models classify the topics and generate documents only using optimizer, we need to look at the words list belong to the topics to guest the topics. This can be a little bit tricky.